In [1]:
#@title License
license = """
Copyright 2022 DeepMind Technologies Limited

Licensed under the Apache License, Version 2.0 (the "License"); you may not
use this file except in compliance with the License. You may obtain a copy of
the License at https://www.apache.org/licenses/LICENSE-2.0. Unless required by
applicable law or agreed to in writing, software distributed under the License
is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
KIND, either express or implied. See the License for the specific language
governing permissions and limitations under the License.
"""
print(license)


Copyright 2022 DeepMind Technologies Limited

Licensed under the Apache License, Version 2.0 (the "License"); you may not
use this file except in compliance with the License. You may obtain a copy of
the License at https://www.apache.org/licenses/LICENSE-2.0. Unless required by
applicable law or agreed to in writing, software distributed under the License
is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
KIND, either express or implied. See the License for the specific language
governing permissions and limitations under the License.



# Dramatron

**Piotr Mirowski, Kory Mathewson, Juliette Love, Jaylen Pittman**, based on an original prototype by **Richard Evans**.

DeepMind, 2022

Dramatron is a script writing tool that leverages large language models. It uses hierarchical generation to ensure long range consistency across the entire script. The work was published as a pre-print in Mirowski, Mathewson et al. (2022) "[Co-Writing Screenplays and Theatre Scripts with Language Models: An Evaluation by Industry Professionals](https://arxiv.org/abs/2209.14958)".

![picture](https://github.com/deepmind/dramatron/raw/main/dramatron_logo.png)

Thanks to everyone who attended the NeurIPS Workshop on Machine Learning for Creativity and Design, and we hope you enjoyed the Dramatron demo. For more information about this project, please see our arXiv paper.

## Citing this work (paper and code):
```bibtex
@article{mirowski2022cowriting,
  title={Co-Writing Screenplays and Theatre Scripts with Language Models: An Evaluation by Industry Professionals},
  author={Mirowski, Piotr and Mathewson, Kory W and Pittman, Jaylen and Evans, Richard},
  journal={arXiv preprint arXiv:2209.14958},
  year={2022}
}
```